In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from urllib.request import urlopen
from IPython.display import clear_output

- 소스보기시 데이터가 없음.
- iframe도 없음.
- 비동기 통신인지 확인 필요.
- 크롬에서 F12를 눌러 개발자 도구를 띄우고 네트워크 탭을 띄워줌.
- 뭔가 변화를 주고 네트워크를 확인했을 때 xhr 이라고 나온다면 비동기 통신임.
- 네트워크 탭에 나와있는 주소를 더블 클릭해서 브라우저에서 주소를 확인.

In [2]:
# 요청 함수
def getSource(site) :
    # 헤더 정보.
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청.
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성.
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
# 하루의 경기 결과를 가져오는 함수
def getData( soup ) : 
    
    data_dict = {
        '팀1 이름' : [],
        '팀2 이름' : [],
        '팀1 투수' : [],
        '팀2 투수' : [],
        '팀1 점수' : [],
        '팀2 점수' : []
    }
    
    
    # 경기 결과 전체를 가져옴.
    a1 = soup.select_one('#todaySchedule')
    # print(a1)
    
    # li tag를 가져옴.
    a2 = a1.select('li.end')
    # print(a2)
    
    
    # li tag 수 만큼 반복.
    for a3 in a2 :
        # 좌측 팀 이름을 가져옴.
        a4 = a3.select_one('div.vs_lft > p > strong')
        data1 = a4.text.strip()
        # print(data1)

        # 우측 팀 이름을 가져옴.
        a5 = a3.select_one('div.vs_rgt > p > strong')
        data2 = a5.text.strip()
        # print(data2)


        # 좌측 팀의 투수 이름을 가져옴.
        a6 = a3.select_one('div.vs_lft > p > span > a')
        # print(a6)
        if a6 != None :
            data3 = a6.text.strip()
        else :
            data3 = np.nan
        # print(data3)    


        # 우측 팀의 투수 이름을 가져옴.
        a7 = a3.select_one('div.vs_rgt > p > span > a')
        if a7 != None :
            data4 = a7.text.strip()
        else :
            data4 = np.nan
            # print(data4)

        # 좌측 팀의 점수를 가져옴.
        a8 = a3.select_one('div.vs_lft > strong')
        data5 = a8.text.strip()
        # print(data5)


        # 우측 팀의 점수를가져옴.
        a9 = a3.select_one('div.vs_rgt > strong')
        data6 = a9.text.strip()
        # print(data6)


        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print(data5)
        # print(data6)

        data_dict['팀1 이름'].append(data1)
        data_dict['팀2 이름'].append(data2)
        data_dict['팀1 투수'].append(data3)
        data_dict['팀2 투수'].append(data4)
        data_dict['팀1 점수'].append(data5)
        data_dict['팀2 점수'].append(data6)
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('baseball.csv') == False :
        # 파일이 없을 경우
        df1.to_csv('baseball.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('baseball.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

In [4]:
# 다음 날짜 값을 가지고 옴.
def getNext( soup ) :
    a1 = soup.select_one('body > div > ul > li:nth-child(6) > a')
    # print(a1)
    
    # onclick 속성을 가져옴.
    a2 = a1.attrs['onclick']
    # print(a2)
    
    
    # 날짜 값을 가져옴.
    a3 = a2.split("'")
    # print(a3)
    return a3[1]

In [5]:
now = '20190313'

while True :
    time.sleep(1)
    clear_output(wait=True)

    site = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={now}&category=kbo'
    
    print(f'현재 {now} 수집중')
    soup = getSource(site)
    getData( soup )
    now = getNext(soup)
    
    if now[:4] == '2020' :
        print('수집완료')
        break;

현재 20191026 수집중
수집완료
